In [1]:
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF,IDF
from pyspark.ml.feature import StringIndexer


from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.mllib.evaluation import MulticlassMetrics
#from pyspark.mllib.evaluation import MultilabelMetrics
#from pyspark.mllib.evaluation import BinaryClassificationMetrics

from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

https://spark.apache.org/docs/latest/api/python/user_guide/index.html

https://github.com/apache/spark

https://github.com/apache/spark/tree/master/examples/src/main/python/mllib

In [2]:
os.environ['JAVA_HOME'] = '/opt/jdk' #Mostra aonde está o JDK

In [123]:
#spark = SparkSession.builder.appName('ml-teste').getOrCreate()

In [3]:
spark = SparkSession.builder.master("local[4]") \
                    .appName('ml-teste') \
                    .getOrCreate()

sc=spark.sparkContext

21/10/28 08:05:49 WARN Utils: Your hostname, abner-Lenovo-G40-70 resolves to a loopback address: 127.0.1.1; using 192.168.15.20 instead (on interface wlp2s0)
21/10/28 08:05:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/28 08:05:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

In [5]:
caminho_df_test = 'data_test'
caminho_df_training = 'data_training'

df_test = spark.read.csv(caminho_df_test,header=True, inferSchema=True)
df_training = spark.read.csv(caminho_df_training, header=True, inferSchema=True)

Exibir colunas de df_test

In [19]:
#df_test.printSchema()

root
 |-- topico: string (nullable = true)
 |-- arquivo: integer (nullable = true)
 |-- texto: string (nullable = true)



Exibir colunas de df_training

In [21]:
#df_training.printSchema()

root
 |-- topico: string (nullable = true)
 |-- arquivo: integer (nullable = true)
 |-- texto: string (nullable = true)



Excluir coluna 'arquivo'

In [53]:
#df_training = df_training.drop('arquivo')
#df_test = df_test.drop('arquivo')

# Parâmetros da Pipeline

In [6]:
label_stringIdx = StringIndexer(inputCol = "topico", outputCol = "label", stringOrderType="alphabetDesc" )
tokenization = Tokenizer(inputCol="texto", outputCol="palavras")
remover_stopword = StopWordsRemover(inputCol="palavras", outputCol="palavras_filtradas")
hashingTF = HashingTF(inputCol="palavras_filtradas", outputCol="tf_features")
idf = IDF(inputCol="tf_features", outputCol="tf_idf_features")

In [ ]:
#pipeline = Pipeline(stages=[label_stringIdx, tokenization, remover_stopword, hashingTF, idf])

#model = pipeline.fit(df_training)

#peek = model.transform(df_training)

#peek.select("palavras_filtradas","tf_features","tf_idf_features").show(n = 30, truncate = 60, vertical = False)

http://spark.apache.org/docs/latest/ml-classification-regression.html

https://spark.apache.org/docs/latest/ml-features#stringindexer

https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StringIndexer.html

https://medium.com/@nutanbhogendrasharma/role-of-stringindexer-and-pipelines-in-pyspark-ml-feature-b79085bb8a6c

# Tratando o df_test

In [17]:
#pipeline = Pipeline(stages=[label_stringIdx, tokenization, remover_stopword, hashingTF, idf])

#model = pipeline.fit(df_test)

#df_test_tratado = model.transform(df_training)

#df_test_tratado.select().show(n = 30, truncate = 60, vertical = False)

#df_test_tratado.printSchema()

# Naive Bayes

https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.NaiveBayes.html

In [95]:
#(trainingData2, testData2) = df_training.randomSplit([0.8, 0.2], seed = 231279) # PARA FIM DE TESTE

In [31]:
nb = NaiveBayes(featuresCol='tf_idf_features', labelCol='label', smoothing=1)
pipelinenb = Pipeline(stages=[label_stringIdx, tokenization, remover_stopword, hashingTF, idf, nb])

nb_model = pipelinenb.fit(df_training) #treina o modelo nb_model

21/10/28 08:39:10 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:39:11 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [ ]:
#nb_model.transform(df_training).show(n = 100, truncate = 30, vertical = False)

## Usando o modelo treinado na base de dados de teste

In [19]:
df_nb = nb_model.transform(df_test)

Exibir dataframe nb_prediction

In [41]:
#df_nb.show()

+------+-------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|topico|arquivo|               texto|label|            palavras|  palavras_filtradas|         tf_features|     tf_idf_features|       rawPrediction|         probability|prediction|
+------+-------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   acq|  11841|dayton hudson not...|  3.0|[dayton, hudson, ...|[dayton, hudson, ...|(262144,[1097,670...|(262144,[1097,670...|[-7332.8593514810...|   [0.0,0.0,0.0,1.0]|       3.0|
|   acq|  11095|maxwell files sui...|  3.0|[maxwell, files, ...|[maxwell, files, ...|(262144,[4562,538...|(262144,[4562,538...|[-13269.208651271...|   [0.0,0.0,0.0,1.0]|       3.0|
|   acq|  11435|freedom to recogn...|  3.0|[freedom, to, rec...|[freedom, recogni...|(262144,[1

21/10/28 09:12:53 WARN DAGScheduler: Broadcasting large task binary with size 12.1 MiB


In [40]:
#df_nb.printSchema()

root
 |-- topico: string (nullable = true)
 |-- arquivo: integer (nullable = true)
 |-- texto: string (nullable = true)
 |-- label: double (nullable = false)
 |-- palavras: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- palavras_filtradas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf_features: vector (nullable = true)
 |-- tf_idf_features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
#df_nb.groupBy('label').count().show()

## Avaliando métricas de classificação de nb_predictions

In [20]:
df_pred_label = df_nb.select(['prediction', 'label']) # Cria um daframe apenas com as colunas predication e label

#df_pred_label.show()


In [ ]:
#df_pred_label.groupBy('label').count().show()

In [19]:
#df_pred_label.write.csv('df_pred_label')

21/10/27 15:16:47 WARN DAGScheduler: Broadcasting large task binary with size 12.2 MiB


In [21]:
rdd_pred_label = df_pred_label.rdd # Converte o dataframe em rdd

In [169]:

#print(rdd_pred_label.collect()) #Mostra todos os dados, tomar cuidado para rdd muito grande
#rdd_pred_label.take(20)


In [ ]:
#df = rdd_pred_label.toDF()
#df.groupBy('label').count().show()

#labels = rdd_pred_label.map(lambda lp: lp.label).distinct().collect()
#for label in labels:
#    print(label)

## Evaluation

In [22]:
metrics = MulticlassMetrics(rdd_pred_label)

21/10/28 08:27:00 WARN DAGScheduler: Broadcasting large task binary with size 12.1 MiB


https://stackoverflow.com/questions/41707370/how-to-calculate-the-f-score-using-multiclassmetrics-in-pyspark

https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html

In [ ]:
# Overall statistics #FUNCIONA SÓ COLOCANDO UMA CLASSE ESPECIFICA
#precision = metrics.precision(2.0)
#recall = metrics.recall(2.0)
#f1Score = metrics.fMeasure(2.0)
#print("Summary Stats")
#print("Precision = %s" % precision)
#print("Recall = %s" % recall)
#print("F1 Score = %s" % f1Score)

In [23]:
#Individual label stats

vec_macroF1 = []
qtdClasses = 0

labels = rdd_pred_label.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("\n\tClass %d precision = %s" % (label, metrics.precision(label)))
    print("\tClass %d recall = %s" % (label, metrics.recall(label)))
    print("\tClass %d F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))
    vec_macroF1.append(metrics.fMeasure(label, beta=1.0))
    qtdClasses +=1
     


21/10/28 08:27:09 WARN DAGScheduler: Broadcasting large task binary with size 12.1 MiB
21/10/28 08:27:10 WARN DAGScheduler: Broadcasting large task binary with size 12.1 MiB



	Class 0 precision = 0.0
	Class 0 recall = 0.0
	Class 0 F1 Measure = 0.0

	Class 1 precision = 0.0
	Class 1 recall = 0.0
	Class 1 F1 Measure = 0.0

	Class 2 precision = 1.0
	Class 2 recall = 0.7857142857142857
	Class 2 F1 Measure = 0.88

	Class 3 precision = 0.9930939226519337
	Class 3 recall = 1.0
	Class 3 F1 Measure = 0.9965349965349966


In [24]:
macroF1 = (sum(vec_macroF1))/qtdClasses
macroF1

0.46913374913374917

In [25]:
print("Accuracy = %s" % metrics.accuracy)

Accuracy = 0.9931972789115646


In [26]:
metrics.confusionMatrix().toArray()

array([[  0.,   0.,   0.,   1.],
       [  0.,   0.,   0.,   1.],
       [  0.,   0.,  11.,   3.],
       [  0.,   0.,   0., 719.]])

## ML Tuning

In [35]:
# Create ParamGrid for Cross Validation
nbparamGrid = ParamGridBuilder() \
               .addGrid(nb.smoothing, [0.0, 0.2, 0.4]) \
               .build()

In [36]:
# Create 5-fold CrossValidator
nbcv = CrossValidator(estimator = pipelinenb,
                      estimatorParamMaps = nbparamGrid,
                      evaluator = MulticlassClassificationEvaluator(predictionCol="prediction"),
                      numFolds = 3)

In [37]:
nbcvModel = nbcv.fit(df_training)


21/10/28 08:54:54 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:54:55 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:54:57 WARN DAGScheduler: Broadcasting large task binary with size 12.1 MiB
21/10/28 08:54:59 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:55:00 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:55:02 WARN DAGScheduler: Broadcasting large task binary with size 12.1 MiB
21/10/28 08:55:04 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:55:05 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:55:08 WARN DAGScheduler: Broadcasting large task binary with size 12.1 MiB
21/10/28 08:55:09 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:55:10 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:55:13 WARN DAGScheduler: Broadcasting l

Py4JJavaError: An error occurred while calling o7938.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 166.0 failed 1 times, most recent failure: Lost task 0.0 in stage 166.0 (TID 4541) (192.168.15.20 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(StringIndexerModel$$Lambda$2814/1846031881: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.ap

21/10/28 08:55:27 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:55:28 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 08:55:29 WARN DAGScheduler: Broadcasting large task binary with size 10.1 MiB
21/10/28 08:55:29 ERROR Executor: Exception in task 0.0 in stage 173.0 (TID 4746)
org.apache.spark.SparkException: Failed to execute user defined function(StringIndexerModel$$Lambda$2814/1846031881: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)


In [57]:
SparkSession.stop(spark)